# Day 18 : Boiling Boulders

Ok, so today we're moving into 3D.
It's all 1unit^3 cubes
We need to find surface area - which is a count of exposed faces

Cube can have 6 sides. Side is exposed if there is no cube in space adjacent to that face.

So a cube in (2,2,2) has potential face-blocking neighbouring cubes at:
(2,2,1)
(2,1,2)
(1,2,2)
(3,2,2)
(2,3,2)
(2,2,3)

Which boils down to: -1 on each of x,y,z then +1 on each of x,y,z



In [2]:
testData = """2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5"""

def process(input:str):
    output = set()
    for l in input.splitlines():
        x,y,z = l.split(',')
        output.add((int(x),int(y),int(z)))
    return output

#test
testCubes = process(testData)
print(testCubes)


{(2, 2, 3), (2, 2, 6), (2, 2, 2), (2, 3, 2), (3, 2, 2), (3, 2, 5), (2, 3, 5), (1, 2, 2), (2, 1, 2), (2, 2, 4), (2, 2, 1), (1, 2, 5), (2, 1, 5)}


In [3]:
neighbours = [  (0,0,1),
                (0,1,0),
                (1,0,0),
                (0,0,-1),
                (0,-1,0),
                (-1,0,0),

            ]
print(neighbours)

import operator

def countSurfaceArea(cubes)->int:
    runningTotal = 0
    for cube in cubes:
        for n in neighbours:
            ncoord = tuple(map(operator.add, cube, n))
            if not ncoord in cubes:
                runningTotal += 1
    return runningTotal

#test
area = countSurfaceArea(testCubes)
print(area)





[(0, 0, 1), (0, 1, 0), (1, 0, 0), (0, 0, -1), (0, -1, 0), (-1, 0, 0)]
64


In [4]:
#puzzle input
puzzle = open('day18input.txt').read()
cubes = process(puzzle)
area = countSurfaceArea(cubes)
print(area)


3500


# Part 2

> Something seems off about your calculation. The cooling rate depends on exterior surface area, but your calculation also included the surface area of air pockets trapped in the lava droplet.

Oh.. this is trickier. 
Pockets of air... might be a maze of a tunnel into it, and wouldn't be a pocket... 
Could I "shave" the shape? 

> Instead, consider only cube sides that could be reached by the water and steam as the lava droplet tumbles into the pond.

Does it offer us a clue? 

What if, instead of considering the solids, we traversed the surface of the shape... 

For any given surface, we have 12 potential surfaces we could traverse to. 
Q:how would we know we've completed the traversal?
Q:where do we start the traversal? How do we know we're not in a pocket? (other than it being unlucky! So might be able to skip this worry for now)

How do we "address those surfaces?"... noting that a surface could be shared by two cubes. So each 3D address is representing 3 unique surfaces (top, left, front.. for exampe=le)
So it's effectively a 4d tuple (x,y,z,face)...  BUT each face is two sided... so it could be air on one side and rock on the other, or vice versa. Doesn't matter if we're only traversing outside!

We need to convert cubes to a surface map. 
wait.. am I missing an easier way of doing this? represent a face a 3D vector normal to the face? Would need position and vector... but might make the maths simplier

e.g. Cube (2,2,2) has 6 faces:
(2,2,2),(0,0,1)
(2,2,2),(0,1,0)
(2,2,2),(1,0,0)
(2,2,2),(0,0,-1)
(2,2,2),(0,-1,0)
(2,2,2),(0,1,0)

Which looks really similar to our "neighbours tranform"

And then we just need a way of identifying neighbouring faces from any given cube.
e.g.

Face (2,2,2),(1,0,0) has 12 possible neighbouring faces:

4 face folding in on the surface: +x on cube, +/- y, z with opposite +/- on face y,z
    (3,2,1),(0,0,1)
    (3,2,3),(0,0,-1)
    (3,1,2),(0,1,0)
    (3,3,2),(0,-1,0)


4 faces in the same direction on neighbours in the plane of the face: +/- y and z on the cube
    (2,3,2),(1,0,0)
    (2,1,2),(1,0,0)
    (2,2,1),(1,0,0)
    (2,2,3),(1,0,0)
4 faces that are adjacent on this cube: +/- on the y and z on the faces
    (2,2,2),(0,1,0)
    (2,2,2),(0,0,1)
    (2,2,2),(0,-1,0)
    (2,2,2),(0,0,-1)
        

How to traverse? brute force... tree of all possible traversals, but never visit the same face twice nor count it twice.


In [5]:
surfaceVectors = [  (0,0,1),
                (0,1,0),
                (1,0,0),
                (0,0,-1),
                (0,-1,0),
                (-1,0,0),

            ]

def surfaces(cubes):
    output = set()
    for cube in cubes:
        for face in surfaceVectors:
            output.add((cube,face))
    return output

#tests
testCubes = process(testData)
print(len(testCubes))
testFaces = surfaces(testCubes)
print(len(testFaces))
print(testFaces)
print(13*6)


13
78
{((3, 2, 2), (1, 0, 0)), ((3, 2, 5), (-1, 0, 0)), ((2, 1, 5), (1, 0, 0)), ((2, 2, 3), (-1, 0, 0)), ((1, 2, 2), (0, -1, 0)), ((2, 2, 4), (0, 0, 1)), ((1, 2, 2), (0, 1, 0)), ((1, 2, 2), (0, 0, -1)), ((3, 2, 5), (1, 0, 0)), ((2, 2, 3), (1, 0, 0)), ((1, 2, 5), (0, -1, 0)), ((2, 2, 1), (-1, 0, 0)), ((1, 2, 5), (0, 1, 0)), ((2, 2, 1), (0, 0, 1)), ((1, 2, 5), (0, 0, -1)), ((2, 2, 6), (1, 0, 0)), ((2, 3, 2), (1, 0, 0)), ((2, 1, 2), (0, 1, 0)), ((2, 1, 2), (0, 0, -1)), ((3, 2, 2), (-1, 0, 0)), ((3, 2, 2), (0, 0, 1)), ((2, 1, 5), (0, 0, 1)), ((2, 3, 5), (0, -1, 0)), ((2, 3, 5), (0, 0, -1)), ((3, 2, 5), (0, 0, 1)), ((2, 2, 3), (0, 0, 1)), ((2, 2, 2), (0, -1, 0)), ((2, 2, 6), (-1, 0, 0)), ((2, 3, 5), (1, 0, 0)), ((1, 2, 2), (-1, 0, 0)), ((2, 2, 2), (0, 1, 0)), ((2, 2, 2), (0, 0, -1)), ((2, 2, 6), (0, 0, 1)), ((2, 3, 2), (-1, 0, 0)), ((2, 3, 2), (0, 0, 1)), ((1, 2, 2), (1, 0, 0)), ((2, 2, 4), (0, -1, 0)), ((2, 1, 2), (0, -1, 0)), ((2, 2, 4), (0, 1, 0)), ((2, 2, 4), (0, 0, -1)), ((1, 2, 5), (1

In [7]:
def outerSurfaces(cubes):
    faces = set()
    for cube in cubes:
        for vect in surfaceVectors:
            faces.add((cube,vect))
    #need to filter out those that have an adjoining cube, so we just get the net of outer surfaces
    output = set()
    for f in faces:
        c, v = f
        ncoord = tuple(map(operator.add, c, v))
        if not ncoord in cubes:
            output.add(f)
    return output


def neighbouringFaces(face):
    #return a set of all the possible neighbouring faces of the given face
    cube, vect = face
    vx,vy,vz = vect
    cx,cy,cz = cube
    neighbouringFaces = set()
    #let's start easy:
    # 4 faces in the same direction on neighbours in the plane of the face: +/- y and z on the cube
#     (2,3,2),(1,0,0)
#     (2,1,2),(1,0,0)
#     (2,2,1),(1,0,0)
#     (2,2,3),(1,0,0)
    cubeTranslates = []
    xTranslates = [(1,0,0),(-1,0,0)]
    yTranslates = [(0,1,0),(0,-1,0)]
    zTranslates = [(0,0,1),(0,0,-1)]
    if abs(vx) == 1: #and yes, i know I could do all of this with matricies
        cubeTranslates.extend(yTranslates)
        cubeTranslates.extend(zTranslates)
    elif abs(vy) == 1:
        cubeTranslates.extend(xTranslates)
        cubeTranslates.extend(zTranslates)
    elif abs(vz) == 1:
        cubeTranslates.extend(xTranslates)
        cubeTranslates.extend(yTranslates)
    for tx, ty, tz in cubeTranslates:
        f = ((cx+tx,cy+ty,cz+tz),vect)
        neighbouringFaces.add(f)
    
    # 4 face folding in on the surface: +x on cube, +/- y, z with opposite +/- on face y,z
    #     (3,2,1),(0,0,1)
    #     (3,2,3),(0,0,-1)
    #     (3,1,2),(0,1,0)
    #     (3,3,2),(0,-1,0)

    #now our translation object also need to be a tupple - maybe should have taken this approach above...
    faceTranslates = []
    xTranslates = [((1,1,0),(0,-1,0)),((1,-1,0),(0,1,0)),((1,0,1),(0,0,-1)),((1,0,-1),(0,0,1))]
    yTranslates = [((1,1,0),(-1,0,0)),((-1,1,0),(1,0,0)),((0,1,1),(0,0,-1)),((0,1,-1),(0,0,1))]
    zTranslates = [((0,1,1),(0,-1,0)),((0,-1,1),(0,1,0)),((1,0,1),(-1,0,0)),((-1,0,1),(1,0,0))]
    if abs(vx) == 1:
        translates = xTranslates
    elif abs(vy) == 1:
        translates = yTranslates
    elif abs(vz) ==1:
        translates = zTranslates
    for (ct,vt) in translates:
        tcx,tcy,tcz = ct
        tvx,tvy,tvz = vt
        f = ((cx+tcx,cy+tcy,cz+tcz),(tvx,tvy,tvz))
        neighbouringFaces.add(f)
    
    # 4 faces that are adjacent on this cube: +/- on the y and z on the faces
    #     (2,2,2),(0,1,0)
    #     (2,2,2),(0,0,1)
    #     (2,2,2),(0,-1,0)
    #     (2,2,2),(0,0,-1) """
    if abs(vx) == 1:
        vTranslates = [(0,1,0),(0,-1,0),(0,0,1),(0,0,-1)]
    elif abs(vy) == 1: 
        vTranslates = [(0,0,1),(0,0,-1),(1,0,0),(-1,0,0)]
    elif abs(vz) == 1:
        vTranslates = [(1,0,0),(-1,0,0),(0,1,0),(0,-1,0)]
    for tvx,tvy,tvz in vTranslates:
        f = (cube,(tvx,tvy,tvz))
        neighbouringFaces.add(f)

    return neighbouringFaces


def outerSurfaceArea(cubes)->int:
    faces = outerSurfaces(cubes)
    print('Total faces: '+str(len(faces)))
    #print(faces)
    runningTotal = 0

    #pick a starting face
    for f in faces:
        break #f now has our starting face.. it's a little non-determinsitic, but we'll see if this works
    searchSpace = set()
    searched = set()
    searchSpace.add(f)
    while len(searchSpace) > 0:
        print('Search space is '+str(len(searchSpace))+' faces. There are '+str(len(faces))+' faces uncounted')
        #print(searchSpace)
        sf = searchSpace.pop()
        if sf in faces and not sf in searched:
            print('Outer face found: '+ str(sf))
            faces.discard(sf)
            runningTotal += 1
            n = neighbouringFaces(sf)
            searchSpace.update(n)
        searched.add(sf)
    return runningTotal

#unit test, to excersie code
f1 = ((3, 2, 2), (1, 0, 0))
ff1 = neighbouringFaces(f1)
print(ff1)
print(len(ff1))

#tests
print('-------')
testCubes = process(testData)
print(outerSurfaceArea(testCubes))

#OK, getting better... but we're still missing 14 faces.

# This hint from the puzzle: The steam will expand to reach as much as possible, completely displacing any air on the outside of the lava droplet but never expanding diagonally.
# OH... this is suggesting taking a volumetric approach... modelling the steam expanding in the total space... and then counting the surfaces of the cubes it joins. TBH, this is less cool that my mesh based approach
# so no need for all the vectorised stuff... that's a shame. 

{((3, 3, 2), (1, 0, 0)), ((3, 2, 2), (0, -1, 0)), ((3, 2, 2), (0, 1, 0)), ((4, 1, 2), (0, 1, 0)), ((3, 2, 2), (0, 0, -1)), ((4, 2, 3), (0, 0, -1)), ((3, 2, 1), (1, 0, 0)), ((3, 2, 2), (0, 0, 1)), ((3, 2, 3), (1, 0, 0)), ((4, 2, 1), (0, 0, 1)), ((3, 1, 2), (1, 0, 0)), ((4, 3, 2), (0, -1, 0))}
12
-------
Total faces: 64
Search space is 1 faces. There are 64 faces uncounted
Outer face found: ((3, 2, 2), (1, 0, 0))
Search space is 12 faces. There are 63 faces uncounted
Search space is 11 faces. There are 63 faces uncounted
Outer face found: ((3, 2, 2), (0, 0, -1))
Search space is 20 faces. There are 62 faces uncounted
Search space is 19 faces. There are 62 faces uncounted
Search space is 18 faces. There are 62 faces uncounted
Search space is 17 faces. There are 62 faces uncounted
Search space is 16 faces. There are 62 faces uncounted
Search space is 15 faces. There are 62 faces uncounted
Search space is 14 faces. There are 62 faces uncounted
Search space is 13 faces. There are 62 faces unc